In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import re
import time
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv')

In [ ]:
data.shape

In [ ]:
data=data.drop('source',axis=1)
data

In [ ]:
unwanted_idx=[]
x=re.compile(r'\d+')
for idx,cols in data.iterrows():
    try:
        if x.match(cols['english_sentence']):
            unwanted_idx.append(idx)
    except:
        print(idx)

In [ ]:
data=data.drop(unwanted_idx + [37554,59804],axis=0)

In [ ]:
eng_d={}
for text in data.english_sentence:
    l=len(text.split(' '))
    eng_d.setdefault(l,0)
    eng_d[l]+=1

In [ ]:
eng_dic={k:v for k,v in sorted(eng_d.items(),key=lambda x: x[1],reverse=True)}

In [ ]:
eng_dic[10]

In [ ]:
hindi_d={}
for text in data.hindi_sentence:
    l=len(text.split(' '))
    hindi_d.setdefault(l,0)
    hindi_d[l]+=1

In [ ]:
hindi_dic={k:v for k,v in sorted(hindi_d.items(),key=lambda x: x[1],reverse=True)}

In [ ]:
hindi_dic[10]

In [ ]:
unwanted_len_id=[]
for idx,cols in data.iterrows():
    if len(cols.english_sentence.split(' ')) > 10 or len(cols.hindi_sentence.split(' ')) > 10:
        unwanted_len_id.append(idx)

In [ ]:
data=data.drop(unwanted_len_id,axis=0)

In [ ]:
data.shape

In [ ]:
train=data.iloc[:25000]
val=data.iloc[25000:]

In [ ]:
train.shape,val.shape

In [ ]:
train=train.to_numpy()
val=val.to_numpy()

In [ ]:
english=train[:,0]
hindi=train[:,1]

In [ ]:
def preprocess_and_tokenize(language):
    pattern=r'[!"#$%&()*,+-./:;\[\]<=>?@\\^_`{|}~\t\n\d“”]'
    language=tf.strings.lower(language)
    language=tf.strings.regex_replace(language,pattern,'')
    language=tf.strings.strip(language)
    
    lang=[]
    for text in language:
        lang.append('<sos> '+ text.numpy().decode('utf-8') + ' <eos>')
            
    lang=np.array(lang)
    tokenizer=keras.preprocessing.text.Tokenizer(filters='',split=' ')
    tokenizer.fit_on_texts(lang)
    tensor=tokenizer.texts_to_sequences(lang)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post',value=0)

    return tensor,tokenizer

In [ ]:
eng_tokenized,eng_tokenizer=preprocess_and_tokenize(english)

In [ ]:
hindi_tokenized,hindi_tokenizer=preprocess_and_tokenize(hindi)

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((tf.Variable(eng_tokenized),tf.Variable(hindi_tokenized))).shuffle(10000).batch(128).prefetch(1)

In [ ]:
class Encoder(keras.Model):
    def __init__(self,vocab_size=10000,emb_dim=128,units=256,batch_size=64):
        super(Encoder,self).__init__()
        self.units = units
        self.batch = batch_size
        self.emb_layer = keras.layers.Embedding(vocab_size,emb_dim)
        self.lstm = keras.layers.LSTM(self.units,return_sequences=True,return_state=True)
        
    def call(self,x,states):
        emb=self.emb_layer(x)
        output,hidden,carry=self.lstm(emb,initial_state=states)
        return output,hidden,carry
    
    def init_hidden_state(self):
        return tf.zeros((self.batch,self.units)),tf.zeros((self.batch,self.units))

In [ ]:
class Decoder(keras.Model):
    def __init__(self,vocab_size=10000,emb_dim=128,units=256,batch_size=64):
        super(Decoder,self).__init__()
        self.units = units
        self.batch = batch_size
        self.emb_layer = keras.layers.Embedding(vocab_size,emb_dim)
        self.lstm = keras.layers.LSTM(self.units,return_sequences=True,return_state=True)
        self.fc=keras.layers.Dense(vocab_size)
        
    def call(self,x,states):
        emb=self.emb_layer(x)
        output,hidden,carry=self.lstm(emb,initial_state=states)
        output=self.fc(output)
        return output,hidden,carry

In [ ]:
optimizer=keras.optimizers.Adam()
loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')
accuracy=keras.metrics.SparseCategoricalAccuracy()
def loss_fn(true,pred):
    mask = tf.math.logical_not(tf.math.equal(true, 0))
    loss_=loss(true,pred)
    mask=tf.cast(mask,dtype=loss_.dtype)
    loss_*=mask
    return tf.reduce_mean(loss_)

def update_accuracy(true,pred):
    accuracy.update_state(true,pred)
    
def get_accuracy():
    accuracy_=accuracy.result().numpy()
    return accuracy_

In [ ]:
enc_vocab_size=len(eng_tokenizer.index_word) + 1
dec_vocab_size=len(hindi_tokenizer.index_word) + 1

In [ ]:
encoder=Encoder(enc_vocab_size,256,512,128)
decoder=Decoder(dec_vocab_size,256,512,128)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
eng_tokenizer.index_word[0]='<pad>'
eng_tokenizer.word_index['<pad>']=0
hindi_tokenizer.index_word[0]='<pad>'
hindi_tokenizer.word_index['<pad>']=0

In [ ]:
@tf.function
def train(input,target,enc_hidden):
    loss__=0.0
    with tf.GradientTape() as tape:
        enc_output,enc_h,enc_c=encoder(input,enc_hidden)
        enc_states=[enc_h,enc_c]
        dec_input=tf.expand_dims(target[:,0],1)
        
        for t in range(1,target.shape[1]):
            dec_output,_,_=decoder(dec_input,enc_states)
            loss__+=loss_fn(target[:,t],dec_output)
            #update_accuracy(target[:,t],dec_output)
            dec_input = tf.expand_dims(target[:, t], 1)
        
    batch_loss=loss__/int(target.shape[1])
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients=tape.gradient(loss__,variables)
        
    optimizer.apply_gradients(zip(gradients,variables))

    return batch_loss

In [ ]:
epochs=50
steps_per_epoch=25000//128 #batches
for epoch in range(epochs):
    start=time.time()
    enc_hidden=encoder.init_hidden_state()
    total_loss=0.0
    for (batch,(inp,tar)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss=train(inp,tar,enc_hidden)
        total_loss+=batch_loss
    #print('Epoch {}      Accuray {:.2f}   Avg. Loss {:.4f}'.format(epoch + 1,float(accuracy.result()),total_loss/steps_per_epoch),end='  ')
    print('Epoch {}      Avg. Loss {:.4f}'.format(epoch + 1,total_loss/steps_per_epoch),end='  ')
    print('Time Taken: {:.1f} sec'.format(time.time() - start))
    accuracy.reset_states()

In [ ]:
def predict(input):
    hidden=[tf.zeros((1,512)),tf.zeros((1,512))]
    _,enc_h,enc_c=encoder(input,hidden)
    enc_states=[enc_h,enc_c]
    result=[]
    dec_input = tf.expand_dims(input[:,0], 0)
    for t in range(input.shape[1]):
        dec_output,_,_=decoder(dec_input,enc_states)
        output_id=tf.math.argmax(dec_output[0],-1)
        output_id=output_id[0].numpy()
        if output_id == hindi_tokenizer.word_index['<eos>']:
            return ' '.join(result)
        dec_input = tf.expand_dims([output_id], 0)
        result.append(hindi_tokenizer.index_word[output_id])

In [ ]:
def preprocess_input(text):
    pattern=r'[!"#$%&()*,+-./:;\[\]<=>?@\\^_`{|}~\t\n\d“”]'
    text=tf.strings.lower(text)
    text=tf.strings.regex_replace(text,pattern,'')
    text=tf.strings.strip(text)
    return text.numpy().decode("utf-8") 

In [ ]:
def get_translation(input_text):
    length=12
    eng_processed=['<sos>',]
    for i in preprocess_input(input_text).split(' '):
        if i != '':
            eng_processed.append(i)
    eng_processed.append('<eos>')
    eng_tokenized=[eng_tokenizer.word_index[i] for i in eng_processed]
    eng_tokenized=tf.keras.preprocessing.sequence.pad_sequences([eng_tokenized],maxlen=length,padding='post',value=0)
    hindi_pred=predict(eng_tokenized)
    print('Pred. Hindi: ',hindi_pred,end='\n\n')

In [ ]:
for i in [1,3,10,15,18,19,21,23,24,26,28,33,34,36,38,45 ]:
    print('No. ',i)
    print('English:     ',english[i])
    print('Hindi:       ',hindi[i])
    get_translation(str(english[i]))

In [ ]:
checkpoint.save(file_prefix = checkpoint_prefix)